In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 4s (64.2 kB/s)
Reading package li

In [2]:
from google.colab import data_table
data_table.enable_dataframe_formatter()
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 228 µs (started: 2022-06-26 04:30:31 +00:00)


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-26 04:30:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-06-26 04:30:31 (11.5 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]

time: 438 ms (started: 2022-06-26 04:30:31 +00:00)


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

time: 8.9 s (started: 2022-06-26 04:30:31 +00:00)


### Load Amazon Reviews Data
[Amazon Review datasets](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt)


In [5]:
### Load Data
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33670092|R1UUISQ1GKOJTI|B00EE62UAE|     583436067|Trademark Home Po...|            Home|          1|            0|          0|   N|                Y|                 Run|Don't buy it clap...| 2015-08-31|
|         US|   13726692|R1HOJ9WE8VCVOD|B001APXO5C|     465035091|O2-Cool 10-Inch P...|            Home|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|   33670092|B00EE62UAE|     583436067|Trademark Home Po...| 2015-08-31|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|   13726692|B001APXO5C|     465035091|O2-Cool 10-Inch P...| 2015-08-31|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|   50131396|B002HFDLCK|     136507891|Hoover Vacuum Cle...| 2015-08-31|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|   16046884|B00PL9EFPQ|     631238459|Banksy Pile of Ri...| 2015-08-31|          2|       

In [7]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28241302|             1|
|   15991768|             1|
|   17067926|             7|
|    8905239|             3|
|   51794064|             1|
|   33132797|             3|
|   21325555|             1|
|    1574612|             1|
|   12819130|             3|
|   44224073|             1|
|   16679709|             1|
|   10742726|             1|
|   52391013|             2|
|   38209321|            10|
|    2545142|             1|
|   52918362|             6|
|   47663761|             3|
|   25031530|            21|
|   29476112|             1|
|   51496479|             1|
+-----------+--------------+
only showing top 20 rows

time: 39.6 s (started: 2022-06-26 04:31:52 +00:00)


In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0486417700|Metallic Foil Ori...|
|1619922363|Cavallini 2014 Fl...|
|1619924307|Cavallini Papers ...|
|1842104497|HUGH JACKMAN 2014...|
|7078976541|VECTRA® 22 Fabric...|
|7482784535|Genuine Leather C...|
|753854514X|Low Voltage 12V A...|
|9178889472|Småland Böna IKEA...|
|9178892171|IKEA - ROTERA Lan...|
|9178909309|IKEA - PORTIS Hat...|
|B00004W40I|King Electrical 6...|
|B00005QFT1|Le Creuset Jumbo ...|
|B00005T3Z7|Endust for Electr...|
|B00006498M|SINGER 6038 46-St...|
|B0000665TD|J-2000 Garment St...|
|B000069BF9|Euro-Pro Shark EP...|
|B00006GXAP|  Ecru Squares Quilt|
|B000075MI0|Stainless Steel C...|
|B000095ZGF|Polder 90616-05 W...|
|B000096P32|Testors Enamel Fi...|
+----------+--------------------+
only showing top 20 rows

time: 50.8 s (started: 2022-06-26 04:32:32 +00:00)


In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1UUISQ1GKOJTI|   33670092|B00EE62UAE|     583436067| 2015-08-31|
|R1HOJ9WE8VCVOD|   13726692|B001APXO5C|     465035091| 2015-08-31|
| RDNGVXMWQN2TN|   50131396|B002HFDLCK|     136507891| 2015-08-31|
|R3OM9COQMVTDJ2|   16046884|B00PL9EFPQ|     631238459| 2015-08-31|
|R3OFUQVR4Y80Q9|   11417282|B00B5TPVQS|     190431573| 2015-08-31|
|R2GD25SBBGRKPO|   48013430|B00NQDGHDC|     124050883| 2015-08-31|
|R3RRSLYKVWH9WB|    7341516|B00I3BYEXM|     933053612| 2015-08-31|
|R2HMFAGJJU6NT3|   20696954|B008QZD7RY|      77515396| 2015-08-31|
|R2QDRZATHBY4GY|   28241302|B00TQ6MXE0|     267158600| 2015-08-31|
|R107946YZK57Q2|   45444347|B004O39RJ4|     473994651| 2015-08-31|
|R31EK6FUI5YAL1|   15080335|B003LZ09C0|     945333576| 2015-08-31|
|R2PCO0R2FKDQLD|   12081067|B00I56KQV4|     383995956| 2015-08

In [11]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1UUISQ1GKOJTI|          1|            0|          0|   N|                Y|
|R1HOJ9WE8VCVOD|          5|            9|          9|   N|                Y|
| RDNGVXMWQN2TN|          5|            0|          0|   N|                Y|
|R3OM9COQMVTDJ2|          2|            0|          0|   N|                Y|
|R3OFUQVR4Y80Q9|          4|            0|          0|   N|                N|
|R2GD25SBBGRKPO|          5|            0|          0|   N|                Y|
|R3RRSLYKVWH9WB|          5|            0|          0|   N|                Y|
|R2HMFAGJJU6NT3|          5|            0|          0|   N|                Y|
|R2QDRZATHBY4GY|          1|            0|          0|   N|                Y|
|R107946YZK57Q2|          2|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://database-bootcamp-16.cc8rspzj5tqc.us-east-1.rds.amazonaws.com:5432/reviews"
config = {"user":"postgres", 
          "password": "", ## password removed
          "driver":"org.postgresql.Driver"}

time: 2.03 ms (started: 2022-06-26 04:33:57 +00:00)


In [14]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

time: 17min 23s (started: 2022-06-26 04:37:33 +00:00)


In [15]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

time: 2min 55s (started: 2022-06-26 04:54:56 +00:00)


In [16]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

time: 5min 20s (started: 2022-06-26 04:57:52 +00:00)


In [17]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

time: 17min 2s (started: 2022-06-26 05:03:13 +00:00)
